In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### retriever 생성 및 툴로 변환

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('data/car.pdf')

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

split_docs = loader.load_and_split(text_splitter)

embedding = OpenAIEmbeddings()

db = FAISS.from_documents(split_docs, embedding)

retriever = db.as_retriever()



In [4]:
from langchain.tools.retriever import create_retriever_tool  # 검색기를 툴로 만드는 기능

retriever_tool = create_retriever_tool(
    retriever,
    name='car_pdf_search',
    description='use this tool to search car information from the PDF document'
)

### 다른 툴과 연동한 Agent 생성

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults()

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4o-mini')

In [9]:
tools = [search_tool, retriever_tool]

In [10]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant."
            "Make sure to use the 'car_pdf_search' toll for searching car information from the PDF document."
            "If you can't find the information from the PDF document, use the 'search_tool' tool for searching information from the web."
        ),
        (
            "placeholder",
            "{chat_history}"
        ),
        (
            "human",
            "{input}"
        ),
        (
            "placeholder",
            "{agent_scratchpad}"
        ),
    ]
)


In [11]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [12]:
from langchain.agents import AgentExecutor

agent_exe = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

result = agent_exe.invoke({'input': '브래이크패드 교체 시기는?'})
result





> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '브래이크패드 교체 시기'}`


[{'title': '자동차 브레이크 라이닝 vs 패드, 교체시기 및 증상, 비용까지!', 'url': 'https://blog.naver.com/commodore11/223734722497', 'content': '브레이크 패드의 교체 주기는 2만km가 일반적이라고 할 수 있습니다. 참고로 브레이크 라이닝은 드럼 브레이크 시스템에서 주로 사용이 되지만 디스크', 'score': 0.8585411}, {'title': '브레이크 패드 교환주기? 라이닝 교환주기? 제동장치에 대한 모든 것!', 'url': 'https://makeittothee.tistory.com/entry/%EB%B8%8C%EB%A0%88%EC%9D%B4%ED%81%AC-%ED%8C%A8%EB%93%9C-%EA%B5%90%ED%99%98%EC%A3%BC%EA%B8%B0-%EB%9D%BC%EC%9D%B4%EB%8B%9D-%EA%B5%90%ED%99%98%EC%A3%BC%EA%B8%B0-%EC%A0%9C%EB%8F%99%EC%9E%A5%EC%B9%98%EC%97%90-%EB%8C%80%ED%95%9C-%EB%AA%A8%EB%93%A0-%EA%B2%83', 'content': '대체적으로 브레이크 패드는 패드 두께 3mm 이하일 때 교체를 하며, 약 20,000km 주행거리를 교체주기로 봅니다. 그리고 라이닝의 경우 라이닝 두께 1mm', 'score': 0.7142662362500001}, {'title': '브레이크 패드 라이닝 교체 시기 비용 직접 확인했습니다 feat.쏘나타', 'url': 'https://m.blog.naver.com/whdgus9640/223195880194', 'content': '보통 브레이크 라이닝은 6-7만 km, 브레이크 패드는 3-4만 km 주기로 교체를 하

{'input': '브래이크패드 교체 시기는?',
 'output': '브레이크 패드의 일반적인 교체 주기는 다음과 같습니다:\n\n- 보통 20,000km를 교체 주기로 보며, 패드 두께가 3mm 이하로 감소했을 때 교체하는 것이 좋습니다.\n- 일부 출처에서는 30,000km에서 40,000km 주기로 교체하는 것을 권장하기도 합니다.\n\n따라서 브레이크 패드는 보통 2만 km에서 4만 km 사이에 교체해주는 것이 바람직합니다. 추가적으로 정기적으로 10,000km마다 점검을 하는 것이 좋습니다. \n\n더 자세한 내용은 [이 링크](https://blog.naver.com/commodore11/223734722497)에서 확인할 수 있습니다.'}

In [13]:
result = agent_exe.invoke({'input': '브래이크패드 교체 시기 pdf에서 찾아줘'})
result




> Entering new AgentExecutor chain...

Invoking: `car_pdf_search` with `{'query': '브래이크패드 교체 시기'}`


시동 및 주행
6-16
다음 지시에 따라 차량을 안전하게 제동하십시
오.
ӝ
 
날씨나 주행 조건에 따라 브레이크 페달을 밟을
때 일시적인 소음이 발생할 수 있지만, 이는 정
상적인 현상입니다.
 
• 브레이크 페달을 여러 번 나누어 밟으십시
오.
- 브레이크를 2-3회에 걸쳐 나누어 밟으면
브레이크 성능이 향상되며, 후방 차량에게
제동 상황을 미리 알려 추돌 사고를 방지
할 수 있습니다.
• 감속이나 정차에 필요한 정도의 적절한 힘으
로 브레이크 페달을 밟으십시오.
• 내리막길에서 브레이크 페달을 장시간 밟으
면 제동 성능이 저하되므로 반드시 엔진 브
레이크와 함께 사용하십시오. 
• 브레이크에 제설제가 묻은 경우, 안전한 장
소에서 브레이크를 밟아 브레이크 디스크와
패드에 묻은 제설제를 제거하십시오. 제설제
로 인해 소음이 발생하거나 타이어가 비정상
적으로 마모될 수 있습니다.
ӝ
 
엔진 브레이크: 물리적인 브레이크는 아니지만
엔진에 브레이크 작용을 하게 하는 것입니다.
엔진의 압축 압력으로 바퀴의 회전을 억제해 제
동력을 얻습니다.
 
디스크 브레이크가 마모된 경우
안전한 주행을 위해 정기적으로 브레이크 패드
의 마모 상태를 점검하십시오.
҃Ҋ
 
• 브레이크 패드가 마모된 상태로 차량을 계속
주행하지 마십시오. 차량의 제동 성능이 떨
어져 심각한 사고가 발생할 수 있습니다.
• 브레이크 패드 교체 시 디스크 안쪽과 바깥
쪽의 패드를 동시에 교체하십시오.
 
• 디스크 브레이크가 마모되어 교체가 필요하
면 브레이크 페달을 밟을 때 쇠가 긁히는 듯
한 소음이 발생합니다.
• 필요한 경우 당사 직영 하이테크센터나 블루
핸즈에서 브레이크 패드 상태를 점검하고 교
체하십시오.
주차 브레이크(EPB)
주차 브레이크(Electronic Parking Brake,
EPB)는 주차 중 

{'input': '브래이크패드 교체 시기 pdf에서 찾아줘',
 'output': '브레이크 패드의 교체에 대한 정보는 다음과 같습니다:\n\n- 브레이크 패드가 마모된 상태로 차량을 계속 주행하지 말아야 하며, 이는 차량의 제동 성능을 떨어뜨려 심각한 사고를 발생시킬 수 있습니다.\n- 브레이크 패드 교체 시, 디스크 안쪽과 바깥쪽의 패드를 동시에 교체하는 것이 중요합니다.\n- 디스크 브레이크가 마모되면, 브레이크 페달을 밟을 때 쇠가 긁히는 소음이 발생할 수 있으므로 주의해야 합니다.\n- 필요시, 직영 하이테크센터나 블루 핸즈에서 브레이크 패드 상태를 점검하고 교체할 수 있습니다.\n\n정기적으로 브레이크 패드의 마모 상태를 점검하는 것이 매우 중요합니다.'}

In [14]:
result = agent_exe.invoke({'input': '카리나 본명 pdf에서 찾아줘'})
result




> Entering new AgentExecutor chain...

Invoking: `car_pdf_search` with `{'query': '카리나 본명'}`


I
색인
0LX3.book  Page 1

지문 인증 
대기 하얀색 -
지문 인증 
성공 파란색 '인증되었습니다. 시동
을 걸 수 있습니다'
지문 인증 
실패 노란색 -
누적 실패
로 인한 
지문 인증 
제한
빨간색
'지문 인증이 제한되었
습니다. X 분 후 다시 시
도하십시오'
ӝ
 
지문 인증 5회 연속 실패 
시의 인증 제한
 
0LX3.book  Page 22

능의 경고등이 켜집니다. 경고 내용은 클러스
터 유틸리티 정보 뷰의 서비스 메시지에서 확인
하실 수 있습니다. 당사 직영 하이테크센터나
블루핸즈에서 점검을 받으십시오.
0LX3.book  Page 5

로운 파일이 생성되면 주차 중 상시와 주차
중 동작감지 중 가장 오래된 파일이 자동 삭
제됩니다.
 
ӝ
 
• 빌트인 캠 시스템에는 GPL, LGPL 및 그 외
오픈소스 라이선스가 적용된 소프트웨어를
포함하고 있습니다.
• 이와 관련한 소스코드를 포함하여 모든 라이
선스 고지사항은 홈페이지
(https://yuracorp.co.kr)를 통해 제공받으
실 수 있습니다.
• 이 제품을 구매한 후 3년 내에 이메일
(opensource@hyundai.com)로 이 제품
에 탑재된 소프트웨어의 오픈소스 코드를 요
청하면 저장 매체 비용, 운반비 등 제공에 필
요한 최소한의 비용을 받고 CD-ROM 등의
저장 매체에 담아 보내드립니다.
 
0LX3.book  Page 82
Invoking: `tavily_search_results_json` with `{'query': '카리나 본명'}`
responded: PDF 문서에서 "카리나 본명"에 대한 정보를 찾을 수 없습니다. 다른 방법으로 정보를 검색해보겠습니다. 잠시만 기다려 주세요.

[{'title': '카리나(동음이의어) - 나무위키', 'url': 'https://namu.wi

{'input': '카리나 본명 pdf에서 찾아줘',
 'output': '카리나의 본명은 "유지민"입니다. 그녀는 걸그룹 에스파의 멤버로, 2000년 4월 11일에 태어났습니다. 더 많은 정보는 [여기](https://namu.wiki/w/%EC%B9%B4%EB%A6%AC%EB%82%98)에서 확인하실 수 있습니다.'}

### 메모리 추가

In [15]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_ids):
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]

agent_history = RunnableWithMessageHistory(
    agent_exe,
    get_session_history,
    input_messages_key='input',
    history_messages_key='chat_history'
)


In [16]:
result = agent_history.invoke(
    {'input': '애플의 최신 휴대폰 기종은?'},
    config={'configurable': {'session_id': 'user123'}}
)

result



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '애플 최신 휴대폰 기종 2023'}`


[{'title': '애플, 2023년 고급 스마트폰 시장 점유율 장악 - Vietnam.vn', 'url': 'https://www.vietnam.vn/ko/apple-chiem-linh-thi-phan-smartphone-cao-cap-nam-2023/', 'content': 'Apple은 2023년 고급 스마트폰 시장 점유율을 장악했습니다. - 사진 2. 아이폰 15 프로와 화웨이 메이트 60 프로. 지난해 최고급 휴대폰을 가장 많이 소비', 'score': 0.75616}, {'title': '2023년, 기대되는 최신 스마트폰 7가지...애플, 삼성, 구글 등', 'url': 'https://www.aseandaily.co.kr/news/articleView.html?idxno=1003959', 'content': 'USB-C 포트가 있는 iPhone 15, 합리적인 가격으로 안드로이드를 실행하는 Pixel 7A, 최고의 야간 촬영 기능을 갖춘 Galaxy S23 Ultra가 올해 출시될', 'score': 0.678789045}, {'title': '2023년 1위는 아이폰14 프로맥스! 삼성 갤럭시S23은? - Naver Blog', 'url': 'https://blog.naver.com/bizzy78/223346760060', 'content': '2023년의 출하량 1~2위가 가장 고가라인업인 아이폰14프로맥스와 아이폰15프로맥스로 쏠린 것을 확인해볼 수 있기도 합니다. 이는 카메라 성능 및 기능성(', 'score': 0.571017072}, {'title': '2023년 출시 스마트폰 정리 - 세티즌', 'url': 'https://m.cetizen.com/review.php?pid=8180&q=view&&qv=rview&pno=

{'input': '애플의 최신 휴대폰 기종은?',
 'chat_history': [],
 'output': '애플의 최신 휴대폰 기종은 **아이폰 15 시리즈**입니다. 특히 **아이폰 15 프로**와 **아이폰 15 프로 맥스** 모델이 포함되어 있습니다. 이 시리즈는 USB-C 포트와 같은 새로운 기능을 도입하며, 고급 스마트폰 시장에서 상위 점유율을 차지하고 있습니다. 자세한 내용은 [여기](https://www.aseandaily.co.kr/news/articleView.html?idxno=1003959)에서 확인하실 수 있습니다.'}

In [17]:
result = agent_history.invoke(
    {'input': '아니, 올해는 2025년이야.'},
    config={'configurable': {'session_id': 'user123'}}
)

result



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '애플 최신 아이폰 기종 2025'}`


[{'title': '2025 Apple Iphone SE4는 apple 역사상 가장 아름답고 저렴한 제품 ...', 'url': 'https://lifeofbaek.tistory.com/485', 'content': '애플이 2025년 3월 말에서 4월 초 사이에 차세대 보급형 스마트폰 iPhone SE 4를 공식 출시할 것으로 예상된다. 이번 SE 4는 애플 역사상 가장 아름답고', 'score': 0.82728493}, {'title': '2025 애플 신제품 로드맵…아이폰17부터 맥북까지 - 네이트 뉴스', 'url': 'https://news.nate.com/view/20250113n22769', 'content': "12일(현지시간) 모바일 매체 폰아레나에 따르면 우선 아이폰17 시리즈에 '에어'라는 새로운 모델이 추가될 예정이다. 아이폰17 에어는 역대 가장 얇은", 'score': 0.6586169777777777}, {'title': '"90Hz 적용 실화냐?!" 아이폰17 & 아이폰 SE4! 2025년 애플의 충격적 ...', 'url': 'https://www.youtube.com/watch?v=QHY5BQSNfTU', 'content': '"90Hz 적용 실화냐?!" 아이폰17 & 아이폰 SE4! 2025년 애플의 충격적 변화 · Comments643.', 'score': 0.54285742}, {'title': 'iPhone 16 - 나무위키', 'url': 'https://namu.wiki/w/iPhone%2016', 'content': '2025년, 고환율이 계속되는데다 아이폰 17시리즈의 가격 인상이 유력하다는 루머가 계속되면서 반대로 아이폰 16의 가성비가 상당히 좋다는 이야기가 나오고 있다.', 'score': 0.4

{'input': '아니, 올해는 2025년이야.',
 'chat_history': [HumanMessage(content='애플의 최신 휴대폰 기종은?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='애플의 최신 휴대폰 기종은 **아이폰 15 시리즈**입니다. 특히 **아이폰 15 프로**와 **아이폰 15 프로 맥스** 모델이 포함되어 있습니다. 이 시리즈는 USB-C 포트와 같은 새로운 기능을 도입하며, 고급 스마트폰 시장에서 상위 점유율을 차지하고 있습니다. 자세한 내용은 [여기](https://www.aseandaily.co.kr/news/articleView.html?idxno=1003959)에서 확인하실 수 있습니다.', additional_kwargs={}, response_metadata={})],
 'output': "2025년의 애플 최신 휴대폰 기종은 **아이폰 SE 4**와 **아이폰 17 시리즈**입니다. \n\n1. **아이폰 SE 4**는 2025년 3월 말에서 4월 초 사이에 출시될 것으로 예상됩니다. 이는 애플 역사상 가장 아름답고 저렴한 보급형 스마트폰으로 기대되고 있습니다. [자세한 내용](https://lifeofbaek.tistory.com/485)에서 확인할 수 있습니다.\n\n2. **아이폰 17** 시리즈도 곧 출시될 예정이며, 이 시리즈에는 '에어'라는 새로운 모델이 추가될 것으로 보입니다. [관련 기사](https://news.nate.com/view/20250113n22769)에서 더 많은 정보를 확인할 수 있습니다. \n\n아이폰 16 시리즈와 관련된 정보도 있으며, 이는 가성비 좋은 모델로 평가받고 있습니다."}